# Load libraries

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier

# Read train and test data

In [32]:
dfXtrain = pd.read_csv('X.train.csv')
dfXtest = pd.read_csv('X.test.csv')
dfYtrain = pd.read_csv('y.train.csv')
Xtrain = np.array(dfXtrain)
Xtest = np.array(dfXtest)

ytrain = np.array(dfYtrain['label'])

In [33]:
print('Data size:', dfXtrain.shape)
dfXtrain.head()

Data size: (4099, 1330)


,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,...,V1322,V1323,V1324,V1325,V1326,V1327,V1328,V1329,V1330,V1331
0,826,0,0,0,0,0,0,NaN,NaN,0,...,0.0,0.0,0,0.0,1.0,1,0.0,NaN,0.0,0.5
1,15,0,0,0,0,0,0,0.030303,0.083333,0,...,0.0,0.0,1,0.0,0.0,0,0.5,0,0.5,0.0
2,242,0,0,0,0,0,0,NaN,NaN,0,...,1.0,1.0,0,0.5,0.0,0,1.0,0,0.0,0.5
3,1038,0,0,0,0,0,0,NaN,NaN,0,...,0.5,0.5,0,0.0,0.0,0,0.0,0,0.0,0.5
4,971,0,0,0,0,0,0,NaN,NaN,0,...,0.0,0.0,1,0.0,0.5,0,0.0,0,0.0,0.0


# Replace all NaN's in train and test data to most frequent values in train data

In [36]:
Xbig = np.vstack((Xtrain, Xtest))

In [37]:
imp = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
imp.fit(Xbig)

Xtrain = imp.transform(Xtrain)
Xtest = imp.transform(Xtest)

# Train Random Forest classifier

In [20]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.5)

In [25]:
from sklearn.metrics import log_loss

In [26]:
def get_score_list(X, y, clf):
    predictions = list()
    score = list()
    for i, tree in enumerate(clf.estimators_):
        predictions.append(tree.predict(X))
        rf_prediction = mean(predictions, axis=0)
        score.append(log_loss(y, rf_prediction))
        if i % 100 == 0:
            print(score[-1])
    return score

In [27]:
def plot_score_graph(clf, X_train, X_test, y_train, y_test, prefix, print_train=True):
    clf.fit(X_train, y_train)
    train_score = get_score_list(X_train, y_train, clf)
    test_score = get_score_list(X_test, y_test, clf)
    if print_train:
        plot(train_score, label=prefix + ' train set error')
    plot(test_score, label=prefix + ' test set error')
    xlabel('n_estimators')
    ylabel('LL')
    legend()

In [ ]:
n_estimators = 1000
plot_score_graph(RandomForestClassifier(n_estimators=n_estimators, criterion='entropy', n_jobs=-1),
               X_train, X_test, y_train, y_test,
               prefix='RF')

1.55080615472
0.0528250632467
0.0537700432704
0.0537150224711
0.0534905275926
0.0532782231726
0.0533019310988
0.0532561407993
0.05328766401
0.0532820099901
4.43112416503
0.242825810861
0.24278661212
0.226872002161
0.226260116477
0.22657808275
0.226038520538
0.226539207487
0.22635059044

600 деревьев пойдёт

In [30]:
clf = RandomForestClassifier(n_estimators=600, criterion='entropy', n_jobs=-1)
clf.fit(Xtrain, ytrain)

ypred = clf.predict_proba(Xtest)[:, 1]

# Save predictions

In [31]:
dfpred = pd.DataFrame()
dfpred['Id'] = np.arange(len(ypred))
dfpred['Prediction'] = ypred

dfpred.to_csv('y.predicted.csv', index = False)